#  Tradução de Textos - Implantação
## Utilização do modelo [MarianMT](https://huggingface.co/transformers/model_doc/marian.html) para tradução. 


* Neste exempo a tradução é feito do inglês para o português, mas ela pode ser feita em qualquer uma das línguas suportadas pelo MarianMT. 
* Para adaptar para traduções em outras línguas é necessário verificacar se há o modelo pré treinado disponível no MarianMT e adaptar o truncamento de strings do [spacy](https://spacy.io/usage/models) para o idioma desejado
* A métrica computada é o [sacrebleu](https://https://github.com/mjpost/sacrebleu) 

### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [6]:
%%writefile Model.py
import joblib
import pandas as pd
import numpy as np
from marianmt_model import MarianMTTranslator


class Model:
    
    def __init__(self):
        
        artifacts = joblib.load("/tmp/data/translate.joblib")
        self.model = artifacts["model"]
    
    
    def predict(self, X, feature_names, meta=None):

        y = X.copy()
        y = self.model.predict(y.tolist())
        
        return y

Overwriting Model.py


In [10]:
#from Model import Model
#model = Model()
#model.model.predict(["Quem sou eu?"])

array(['Who am I?'], dtype='<U9')